In this example, you are going to assessing car damage using computer vision capabilities of FMS to automate insurance claims.

In the following cell, we are importing necessary packages and setting up a bedrock client.

In [42]:
import json
import os
import sys

import boto3
import botocore

boto3_bedrock = boto3.client('bedrock-runtime')

In the cell below, we are reading the image file and converting it to base64-encoded string.

In [43]:
#import image locally, read it and convert it into base64
import base64
from PIL import Image

# Convert the image to base64 format
with open("<PATH_TO_THE_IMAGE>", "rb") as f:
    encoded_image = base64.b64encode(f.read()).decode('utf-8')

In the prompt instruction below, we are giving a persona to the model and providing instructions on the task they need to perform.

In [44]:
prompt = """
You are an assistant to insurance claims agents. Your job is to provide an assessment in three categories - minor damage, moderate damage, and major damage. You can provide additional details such as what part of the vehicle is damage, what kind of repair will likely be require, and approximate average cost. Be reasonable, accurate, and do not inflate or deflate the extent of damage. 

Please provide your assessment for the supplied image.
"""

Creating request body according to the model requirement.

In [45]:
body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1000,
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "image",
            "source": {
              "type": "base64",
              "media_type": "image/jpeg",
              "data": encoded_image
            }
          },
          {
              "type": "text", 
              "text": prompt
          }
        ]
      }
    ],
    "temperature":0.5,
    "top_k":250,
    "top_p":0.5,
    "stop_sequences":[]
})

Invoking Claude 3 Sonnet model and supplying the prepared prompt along with the image as part of the request body.

In [46]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
accept = 'application/json'
contentType = 'application/json'

try:

    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    #printing the generated response from the model.
    print(response_body.get('content')[0]['text'])
    
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

Based on the image provided, I would assess the damage to the vehicle as follows:

Major Damage:
- The front end of the vehicle has sustained significant impact damage. Both the hood and front bumper appear to be heavily crumpled and deformed, indicating a high-energy collision.
- There is likely structural damage to the front frame rails and other components that provide structural integrity to the vehicle's front end.

Moderate Damage:
- The headlight assemblies on both sides appear to be broken or dislodged from their housings.
- Depending on the extent of the impact, there may be moderate damage to the radiator, cooling system components, and other mechanical parts located in the engine compartment.

Minor Damage:
- The windshield seems to be intact, but there may be minor cracks or chips that need to be addressed.
- The side mirrors and other exterior trim pieces may have sustained minor scratches or dents.

Repair Considerations:
Given the severity of the front-end damage, it is 